<a href="https://colab.research.google.com/github/akansh12/CS6910_Assignment/blob/main/Part_A_Akansh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import torchvision
import os
import torch.nn as nn
from torch.nn import functional as F

In [2]:
import matplotlib.pyplot as plt
import copy
from tqdm.notebook import tqdm

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Qurstion-1

In [3]:
### HELPER FUNCTIONS
def findConv2dOutShape(H_in,W_in,conv,pool=2):
  kernel_size=conv.kernel_size
  stride=conv.stride
  padding=conv.padding
  dilation=conv.dilation

  H_out=np.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
  W_out=np.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
  if pool:
    H_out/=pool
    W_out/=pool
  return int(H_out),int(W_out)

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = "cpu"

In [4]:
class conv_net(nn.Module):
  def __init__(self, params): #num_filters, filter_size, activation_functions,dense_neurons,num_outputs
    super(conv_net,self).__init__()
    c_in, h_in, w_in = params["input_shape"]
    num_filters = params["num_filters"]
    filter_size = params["filter_size"]
    self.act_function = params["activation_functions"]
    dense_neurons = params["dense_neurons"]
    num_output = params["num_output"]
    self.dropout = params["dropout"]


    ####Conv layers
    self.conv1 = nn.Conv2d(c_in, num_filters[0], kernel_size=filter_size[0])
    h,w = findConv2dOutShape(h_in, w_in, self.conv1)

    self.conv2 = nn.Conv2d(num_filters[0], num_filters[1], kernel_size=filter_size[1])
    h,w = findConv2dOutShape(h, w, self.conv2)

    self.conv3 = nn.Conv2d(num_filters[1], num_filters[2], kernel_size=filter_size[2])
    h,w = findConv2dOutShape(h, w, self.conv3)


    self.conv4 = nn.Conv2d(num_filters[2], num_filters[3], kernel_size=filter_size[3])
    h,w = findConv2dOutShape(h, w, self.conv4)


    self.conv5 = nn.Conv2d(num_filters[3], num_filters[4], kernel_size=filter_size[4])
    h,w = findConv2dOutShape(h, w, self.conv5)
    #FC layers
    self.num_flatten = h*w*num_filters[4]
    self.fc1 = nn.Linear(self.num_flatten, dense_neurons)
    self.fc2 = nn.Linear(dense_neurons, num_output)

  def forward(self,x):
    x = self.act_function[0](self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[1](self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[2](self.conv3(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[3](self.conv4(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[4](self.conv5(x))
    x = F.max_pool2d(x, 2, 2)
    #Flatening the layers
    x = x.view(-1, self.num_flatten)
    
    x = self.act_function[5](self.fc1(x))
    x = F.dropout(x, self.dropout, training= self.training)
    x = self.fc2(x)
    x = F.log_softmax(x, dim=1)
    return x   


In [5]:
params = {"input_shape": (3,224,224),
          "num_filters": [8,16,32,64,128],
          "filter_size":[3,3,3,3,3],
          "activation_functions": [nn.functional.relu]*6,
          "dense_neurons": 128,
          "num_output":10,
          "dropout": 0.2          
          }

In [6]:
model = conv_net(params)
model.to(device);

1. What is the total number of computations done by your network? (assume m filters in each layer of size k×k and n neurons in the dense layer)

- Ans: 

2. What is the total number of parameters in your network? (assume m filters in each layer of size k×k and n neurons in the dense layer)

- Ans: 

### Question 2

In [18]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import os
path2data_train="/content/drive/MyDrive/inaturalist_12K/train"
path2data_test = "/content/drive/MyDrive/inaturalist_12K/val"

In [19]:
from torchvision.transforms.transforms import RandomRotation
# DATA-Augmentations
train_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.RandomRotation((-20,20)),
transforms.RandomHorizontalFlip(p=0.5),
transforms.RandomVerticalFlip(p=0.5),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [20]:
data = datasets.ImageFolder(path2data_train, train_transforms)
test_data = datasets.ImageFolder(path2data_test, test_transforms)

In [21]:
print(data.class_to_idx)

{'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}


In [22]:
### Spiliting the data into train-val
n_val = int(np.floor(0.1 * len(data)))
n_train = len(data) - n_val
train_ds, val_ds = random_split(data, [n_train, n_val])

In [23]:
print("Number of datapoints in train: ", len(train_ds))
print("Number of datapoints in val: ", len(val_ds))
print("Number of datapoints in test: ", len(test_data))

Number of datapoints in train:  9000
Number of datapoints in val:  999
Number of datapoints in test:  2000


In [24]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=False)
test_dl = DataLoader(test_data, batch_size=64, shuffle=False)

In [26]:
def accuracy(y_pred,y_true):
    y_pred = torch.exp(y_pred)
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

for param in model.parameters():
    param.require_grad = True

from torch import optim

opt = optim.Adam(model.parameters(), lr=1e-3)
model = model.to(device)
loss_func = nn.NLLLoss()
schedular = optim.lr_scheduler.ReduceLROnPlateau(opt,factor = 0.1,patience = 5)


In [34]:
def training(model, train_params):
  epochs = train_params["num_epochs"]
  train_dl = train_params["train_dl"]
  loss_func = train_params["loss_func"]
  opt = train_params["opt"]
  val_dl = train_params["val_dl"]

  train_loss_hist = []
  valid_loss_hist = []
  train_acc_hist = []
  valid_acc_hist = []
  valid_loss_min = np.Inf

  for i in range(epochs):
      
      train_loss = 0.0
      valid_loss = 0.0
      train_acc = 0.0
      valid_acc = 0.0 
      
      
      model.train()
      
      for images,labels in tqdm(train_dl):
          
          images = images.to(device)
          labels = labels.to(device)
          
          ps = model(images)
          loss = loss_func(ps,labels)
          
          opt.zero_grad()
          loss.backward()
          opt.step()
          
          train_acc += accuracy(ps,labels)
          train_loss += loss.item()
          
      avg_train_acc = train_acc / len(train_dl)
      train_acc_hist.append(avg_train_acc)
      avg_train_loss = train_loss / len(train_dl)
      train_loss_hist.append(avg_train_loss)
          
      model.eval()
      with torch.no_grad():
          
          for images,labels in tqdm(val_dl):
              
              images = images.to(device)
              labels = labels.to(device)
              
              ps = model(images)
              loss = loss_func(ps,labels)
              
              valid_acc += accuracy(ps,labels)
              valid_loss += loss.item()
              
              
          avg_valid_acc = valid_acc / len(val_dl)
          valid_acc_hist.append(avg_valid_acc)
          avg_valid_loss = valid_loss / len(val_dl)
          valid_loss_hist.append(avg_valid_loss)
          
          schedular.step(avg_valid_loss)
          
          if avg_valid_loss <= valid_loss_min:
              print('Validation loss decreased ({:.6f} --> {:.6f}).   Saving model ...'.format(valid_loss_min,avg_valid_loss))
              torch.save({
                  'epoch' : i,
                  'model_state_dict' : model.state_dict(),
                  'optimizer_state_dict' : opt.state_dict(),
                  'valid_loss_min' : avg_valid_loss
              },'model.pth')
              
              valid_loss_min = avg_valid_loss
              
      print("Epoch : {} Train Loss : {:.6f} Train Acc : {:.6f}".format(i+1,avg_train_loss,avg_train_acc))
      print("Epoch : {} Valid Loss : {:.6f} Valid Acc : {:.6f}".format(i+1,avg_valid_loss,avg_valid_acc))

  return model.load_state_dict(torch.load("/content/model.pth")['model_state_dict'])


In [35]:
train_params = {"num_epochs": 30,
                "loss_func": loss_func,
                "train_dl":train_dl ,
                "val_dl":val_dl, 
                "test_dl": test_dl,
                "path2weights": "./",
                "opt": opt

               }

In [ ]:
training(model, train_params)

 24%|██▍       | 69/282 [00:50<02:35,  1.37it/s]

In [33]:
model.load_state_dict(torch.load("/content/model.pth")['model_state_dict'])

<All keys matched successfully>

In [ ]:
from PIL import Image
def preprocess_image(path2image):
    image =Image.open(path2image)

    preprocess = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

    image = preprocess(image)
    image.unsqueeze_(0)

    return image

rnd_image = preprocess_image("/content/drive/MyDrive/inaturalist_12K/val/Animalia/016125d0354a1fc95ed911c85ddec844.jpg")
model = model.to(device)
rnd_image = rnd_image.to(device)

no_of_layers=0
conv_layers=[]

model_children=list(model.children())

for child in model_children:
  if type(child)==nn.Conv2d:
    no_of_layers+=1
    conv_layers.append(child)
  elif type(child)==nn.Sequential:
    for layer in child.children():
      if type(layer)==nn.Conv2d:
        no_of_layers+=1
        conv_layers.append(layer)


first_layer_output = conv_layers[0](rnd_image)
n_row = 2
n_col = 4

fig, axs = plt.subplots(n_row,n_col, figsize = (10,10))
f_count = 0
for i in range(n_row):
  for j in range(n_col):
    axs[i,j].imshow(first_layer_output[0][f_count].cpu().detach().numpy(), cmap = "gray")
    f_count += 1
